# Another implementation of word2vec: this is the first version that I saw

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import inspect
import string
import time

In [2]:
filename = 'pt96.txt'  
file_path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
file_path = os.path.join(file_path,'data')
file_path = os.path.join(file_path,filename)


def read_data(file_path):
    dic_trans = {key: None for key in string.punctuation}
    translator = str.maketrans(dic_trans)
    words = []
    with open(file_path) as inputfile:
        for line in inputfile:
            aux = line.translate(translator)
            words.extend(aux.strip().split())
    return words

words = read_data(file_path)
print('Data size', len(words))

Data size 15164437


In [3]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 1077275], ('de', 763544), ('e', 422852), ('a', 422319), ('o', 335045)]
Sample data [7816, 18649, 0, 5525, 18649, 0, 9954, 6502, 8545, 2190]


In [4]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(int(batch_size / num_skips)):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], '->', labels[i, 0])
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

18649 -> 7816
Contos -> Conto
18649 -> 0
Contos -> UNK
0 -> 5525
UNK -> 1870
0 -> 18649
UNK -> Contos
5525 -> 0
1870 -> UNK
5525 -> 18649
1870 -> Contos
18649 -> 0
Contos -> UNK
18649 -> 5525
Contos -> 1870


In [5]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
    # Variables.
    embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases,train_labels, embed,num_sampled, vocabulary_size))

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [6]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  ts = time.time()
  print("Initialized")
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step", step, ":", average_loss)
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = "Nearest to %s:" % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = "%s %s," % (log, close_word)
        print(log)
  te = time.time()
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0 : 7.69314575195
Nearest to são: Rebels, Calgary, menção, expulsou, estruturados, Evangelho, meigo, incorreto,
Nearest to dia: Celeiro, progressivamente, Ming, Imóvel, evangelísticas, Dear, ogã, Fuentes,
Nearest to outros: portaria, Lashly, Speedy, calcário, amargo, Toulouse, superficiais, Lamar,
Nearest to mas: recuo, caíra, THX, arrabaldes, PowerShell, confeccionado, suspira, Suárez,
Nearest to sem: cortadas, Satanás, 193233, Srs, Mellão, satisfatória, Eru, cirúrgicos,
Nearest to por: colorada, rodoviário, despeito, criminalidade, Albright, ensaiando, albanesa, inferior,
Nearest to sobre: protetora, Confeiteiros, blogs, réu, caminhos, empresa, confessou, Sagrado,
Nearest to depois: antena, cortejo, Shanks, agravamento, Shinano, electro, haviase, sentavase,
Nearest to de: parecida, Options, plásticos, rígidas, aluno, islâmica, Fornost, Borgo,
Nearest to que: adivinhar, Carl, sociedades, Yves, eventualmente, Jena, itá, Marajoara,
Nearest to pela: sonho

In [7]:
print("duration= ", te-ts)

duration=  208.2268509864807


In [8]:
from scipy import spatial

def analogy(w1,w2,w3,index_to_word,word2index, embeddings):
    a = embeddings[word2index[w1]]
    b = embeddings[word2index[w2]]
    c = embeddings[word2index[w3]]
    def apply_dot(x):
        return b.dot(x) - a.dot(x) + c.dot(x)
    all_results = [(apply_dot(w), index) for index, w in enumerate(embeddings) if (index!= word2index[w1] and index!= word2index[w2] and index!= word2index[w3])]
    all_results.sort(reverse=True)
    result = [(index_to_word[index],value) for (value,index) in all_results[0:11]]
    return result[0][0]  

def top_k_sim(vector, index_to_word, embeddings,k=10):
    all_sim = [(1 - spatial.distance.cosine(vector, w),index) for index, w in enumerate(embeddings)]
    all_sim.sort(reverse =True)
    result = [(index_to_word[index],value) for (value,index) in all_sim[0:k+1]]
    return result 

In [9]:
mulher = final_embeddings[dictionary['mulher']]
top_k_sim(mulher,reverse_dictionary,final_embeddings)

[('mulher', 0.99999999999999645),
 ('moça', 0.41701802116739883),
 ('organização', 0.40576920474951528),
 ('faixa', 0.37397059047046843),
 ('selecionados', 0.36919378454648166),
 ('Rivers', 0.36492689638894182),
 ('vila', 0.36033193109441464),
 ('esposa', 0.35712901609367287),
 ('pessoa', 0.35574505592912353),
 ('DECSYSTEM20', 0.35382746778890306),
 ('declaração', 0.34741882399208657)]

In [12]:
eval_file="AnalogiesBr_little.txt"
vocab = set(dictionary.keys())
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
prefix = os.path.join(currentdir, "evaluation")
file_path = os.path.join(prefix, eval_file)
valid_tests = 0
correct_answer = 0
total_lines = 0
total_loss = 0


In [13]:
import time
import sys
initial_time = time.time()
with open(file_path) as inputfile:
    for line in inputfile:
        total_lines += 1
        initial = time.time()
        list_line = line.strip().split()
        if all([word in vocab for word in list_line]):
            valid_tests += 1
            analogue = analogy(list_line[0],
                               list_line[1],
                               list_line[2],
                               reverse_dictionary,
                               dictionary,
                               final_embeddings)
            print("\nAnalogy -->", list_line)
            print("prediction -->", analogue)
            if analogue == list_line[3]:
                correct_answer += 1
                print("\nYESSSSSSSSSSSSSSSSSSSSS\n")
            current_time = time.time() - initial
            sys.stdout.write('\rcurrent_line:{}, duration = {}'.format(total_lines,
                                                                        current_time))
            sys.stdout.flush()

print("\ntotal_lines = {}".format(total_lines))
print("valid_tests = {}".format(valid_tests))
print("correct_answer = {}".format(correct_answer))
print("total_loss = {}".format(total_loss))
print("duration = {}".format(time.time() - initial_time))



Analogy --> ['rapaz', 'moça', 'irmão', 'irmã']
prediction --> estritos
current_line:2, duration = 0.1867198944091797
Analogy --> ['rapaz', 'moça', 'irmãos', 'irmãs']
prediction --> luademel
current_line:3, duration = 0.17479562759399414
Analogy --> ['rapaz', 'moça', 'pai', 'mãe']
prediction --> polícia
current_line:4, duration = 0.17296195030212402
Analogy --> ['rapaz', 'moça', 'pai', 'mãe']
prediction --> polícia
current_line:5, duration = 0.17183136940002441
Analogy --> ['rapaz', 'moça', 'avô', 'avó']
prediction --> Sox
current_line:6, duration = 0.17393040657043457
Analogy --> ['rapaz', 'moça', 'avô', 'avó']
prediction --> Sox
current_line:7, duration = 0.17398858070373535
Analogy --> ['rapaz', 'moça', 'neto', 'neta']
prediction --> Lax
current_line:8, duration = 0.18085622787475586
Analogy --> ['rapaz', 'moça', 'noivo', 'noiva']
prediction --> Sox
current_line:9, duration = 0.17641186714172363
Analogy --> ['rapaz', 'moça', 'ele', 'ela']
prediction --> ela

YESSSSSSSSSSSSSSSSSSSSS


In [ ]:
reverse_dictionary[414]